In [59]:
# Pull traffic data from Chicago City Data API
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import numpy as np

# Import downloaded csv file
crashes_csv = "Traffic_Crashes.csv"
traffic_crashes = pd.read_csv(crashes_csv)
crashes_df = pd.DataFrame(traffic_crashes)
crashes_drop_df = crashes_df.dropna(subset=['LATITUDE', 'LONGITUDE'], how='all')
crashes_drop_df.head()

C:\Users\gripei01\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,JC319175,NaN,6/24/2019 1:30,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,...,0.0,0.0,1.0,0.0,1,2,6,41.950021,-87.648729,POINT (-87.648729461363 41.950021042777)
1,JC319110,NaN,6/24/2019 0:15,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,...,0.0,0.0,1.0,0.0,0,2,6,41.800297,-87.706349,POINT (-87.706348495564 41.800297179753)
2,JC319103,NaN,6/24/2019 0:09,25,NO CONTROLS,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,ONE-WAY,...,0.0,0.0,2.0,0.0,0,2,6,41.878969,-87.726493,POINT (-87.726492697013 41.878968897127)
3,JC319098,NaN,6/24/2019 0:05,30,TRAFFIC SIGNAL,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,...,0.0,0.0,2.0,0.0,0,2,6,41.953187,-87.757266,POINT (-87.757265734676 41.953186849643)
4,JC319100,NaN,6/23/2019 23:50,30,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN BARRIER,...,0.0,0.0,2.0,0.0,23,1,6,41.867551,-87.625209,POINT (-87.625209100215 41.867550959738)


In [60]:
crashes_drop_df.columns

Index(['RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'LANE_CNT',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE',
       'CRASH_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I',
       'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LATITUDE', 'LONGITUDE

In [61]:
# Reduce to only columns we care about
crashes_drop_df = crashes_drop_df[['CRASH_DATE', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
                                   'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'FIRST_CRASH_TYPE','PRIM_CONTRIBUTORY_CAUSE',
                                   'INJURIES_TOTAL','INJURIES_FATAL', 'CRASH_HOUR','CRASH_DAY_OF_WEEK',
                                   'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION']]


# Reformat the date column to be easily comparable dates
crashes_drop_df['CRASH_DATE'] = pd.to_datetime(crashes_drop_df['CRASH_DATE'])
crashes_drop_df.sort_values(by='CRASH_DATE', ascending=False)

# Cut Lat and Lon to 4 decimal places
# crashes_drop_df['LATITUDE'] = crashes_drop_df['LATITUDE'].round(3)
# crashes_drop_df['LONGITUDE'] = crashes_drop_df['LONGITUDE'].round(3)


# Filter the data frame for only dates 2018 onward

crashes_2017_18 = crashes_drop_df.loc[(crashes_drop_df['CRASH_DATE'] >= "2017-01-01") & 
                                        (crashes_drop_df['CRASH_DATE'] <= "2018-12-31"), :]

crashes_2017_18.head()


,CRASH_DATE,WEATHER_CONDITION,LIGHTING_CONDITION,STREET_NO,STREET_DIRECTION,STREET_NAME,FIRST_CRASH_TYPE,PRIM_CONTRIBUTORY_CAUSE,INJURIES_TOTAL,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
55419,2018-12-31 00:00:00,CLEAR,DARKNESS,6350,N,HOYNE AVE,REAR END,FOLLOWING TOO CLOSELY,0.0,0.0,0,2,12,41.997549,-87.682706,POINT (-87.682705990061 41.997548692976)
55420,2018-12-30 23:40:00,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",1958,W,FULLERTON AVE,REAR END,FOLLOWING TOO CLOSELY,0.0,0.0,23,1,12,41.925136,-87.677879,POINT (-87.677878977271 41.92513614134)
55421,2018-12-30 23:30:00,CLEAR,"DARKNESS, LIGHTED ROAD",3431,N,AUSTIN AVE,FIXED OBJECT,DISTRACTION - FROM INSIDE VEHICLE,1.0,0.0,23,1,12,41.943075,-87.776250,POINT (-87.776249727669 41.943074658434)
55422,2018-12-30 23:20:00,CLEAR,"DARKNESS, LIGHTED ROAD",3735,N,MILWAUKEE AVE,REAR END,FOLLOWING TOO CLOSELY,0.0,0.0,23,1,12,41.948788,-87.739519,POINT (-87.739518820234 41.948787810242)
55423,2018-12-30 23:15:00,CLEAR,DARKNESS,8900,S,STONY ISLAND AVE,SIDESWIPE SAME DIRECTION,EXCEEDING AUTHORIZED SPEED LIMIT,0.0,0.0,23,1,12,41.733188,-87.585713,POINT (-87.585712697936 41.733187965898)


In [71]:
# Make a data frame with the Chicago population from 2015 to 2019

pop_years = [2014, 2015, 2016, 2017, 2018]
chi_pops = [2728524, 2726215, 2718946, 2713067, 2705994]
chi_yearly_pops = {"Year": pop_years, "Chicago Population": chi_pops}

chicago_pop_df = pd.DataFrame(chi_yearly_pops)
chicago_pop_df

,Year,Chicago Population
0,2014,2728524
1,2015,2726215
2,2016,2718946
3,2017,2713067
4,2018,2705994


In [73]:
# Filter original dataframe for only crashes with a fatality
fatal_crashes = crashes_2017_18.loc[(crashes_2017_18['INJURIES_FATAL'] > 0), :]

fatal_groupby = fatal_crashes.groupby(['FIRST_CRASH_TYPE', 'LIGHTING_CONDITION'])
fatal_groupby.count().sort_values(by='Crash Date', ascending=False).head(10)

Crash Date  Crash Time  \
FIRST_CRASH_TYPE     LIGHTING_CONDITION                               
FIXED OBJECT         DARKNESS, LIGHTED ROAD          30          30   
PEDESTRIAN           DARKNESS, LIGHTED ROAD          22          22   
FIXED OBJECT         DAYLIGHT                        18          18   
PARKED MOTOR VEHICLE DARKNESS, LIGHTED ROAD          17          17   
PEDESTRIAN           DAYLIGHT                        14          14   
PARKED MOTOR VEHICLE DAYLIGHT                        11          11   
ANGLE                DAYLIGHT                         7           7   
                     DARKNESS, LIGHTED ROAD           7           7   
TURNING              DAYLIGHT                         6           6   
HEAD ON              DARKNESS, LIGHTED ROAD           6           6   

                                             WEATHER_CONDITION  STREET_NO  \
FIRST_CRASH_TYPE     LIGHTING_CONDITION                                     
FIXED OBJECT         DARKNESS, LIGHTED ROAD                 30         30   
PEDESTRIAN           DARKNESS, LIGHTED ROAD                 22         22   
FIXED OBJECT         DAYLIGHT                               18         18   
PARKED MOTOR VEHICLE DARKNESS, LIGHTED ROAD                 17         17   
PEDESTRIAN           DAYLIGHT                               14         14   
PARKED MOTOR VEHICLE DAYLIGHT                               11         11   
ANGLE                DAYLIGHT                                7          7   
                     DARKNESS, LIGHTED ROAD                  7          7   
TURNING              DAYLIGHT                                6          6   
HEAD ON              DARKNESS, LIGHTED ROAD                  6          6   

                                             STREET_DIRECTION  STREET_NAME  \
FIRST_CRASH_TYPE     LIGHTING_CONDITION                                      
FIXED OBJECT         DARKNESS, LIGHTED ROAD                30           30   
PEDESTRIAN           DARKNESS, LIGHTED ROAD                22           22   
FIXED OBJECT         DAYLIGHT                              18           18   
PARKED MOTOR VEHICLE DARKNESS, LIGHTED ROAD                17           17   
PEDESTRIAN           DAYLIGHT                              14           14   
PARKED MOTOR VEHICLE DAYLIGHT                              11           11   
ANGLE                DAYLIGHT                               7            7   
                     DARKNESS, LIGHTED ROAD                 7            7   
TURNING              DAYLIGHT                               6            6   
HEAD ON              DARKNESS, LIGHTED ROAD                 6            6   

                                             PRIM_CONTRIBUTORY_CAUSE  \
FIRST_CRASH_TYPE     LIGHTING_CONDITION                                
FIXED OBJECT         DARKNESS, LIGHTED ROAD                       30   
PEDESTRIAN           DARKNESS, LIGHTED ROAD                       22   
FIXED OBJECT         DAYLIGHT                                     18   
PARKED MOTOR VEHICLE DARKNESS, LIGHTED ROAD                       17   
PEDESTRIAN           DAYLIGHT                                     14   
PARKED MOTOR VEHICLE DAYLIGHT                                     11   
ANGLE                DAYLIGHT                                      7   
                     DARKNESS, LIGHTED ROAD                        7   
TURNING              DAYLIGHT                                      6   
HEAD ON              DARKNESS, LIGHTED ROAD                        6   

                                             INJURIES_TOTAL  INJURIES_FATAL  \
FIRST_CRASH_TYPE     LIGHTING_CONDITION                                       
FIXED OBJECT         DARKNESS, LIGHTED ROAD              30              30   
PEDESTRIAN           DARKNESS, LIGHTED ROAD              22              22   
FIXED OBJECT         DAYLIGHT                            18              18   
PARKED MOTOR VEHICLE DARKNESS, LIGHTED ROAD              17           

In [75]:
# Import Rideshare CSV
rideshare_csv = "rs_filter_csv.csv"
rideshare_data = pd.read_csv(rideshare_csv)
rideshare_df = pd.DataFrame(rideshare_data)

rideshare_df = rideshare_df[['Month', 'NUMBER_OF_TRIPS_2015', 'NUMBER_OF_TRIPS_2016', 'NUMBER_OF_TRIPS_2017',
                            'NUMBER_OF_TRIPS_2018']]

rideshare_df

,Month,NUMBER_OF_TRIPS_2015,NUMBER_OF_TRIPS_2016,NUMBER_OF_TRIPS_2017,NUMBER_OF_TRIPS_2018
0,1,518539,1733583,3582540,4842911
1,2,518539,1732573,3455601,4718735
2,3,712521,1887034,4017804,5284216
3,4,764487,2014292,3839776,3500907
4,5,868650,2184055,4039394,3580748
5,6,876603,2256966,4007753,5292651
6,7,956447,2690828,4109970,5052784
7,8,1041740,2764145,4129588,5236080
8,9,1098735,2813310,4084594,4981605
9,10,1140334,3250032,4523491,5230188


In [89]:
# Round crashes_2017_18 to three decimals, and merge with speed cam data

crashes_2017_18['LATITUDE'] = crashes_2017_18['LATITUDE'].round(3)
crashes_2017_18['LONGITUDE'] = crashes_2017_18['LONGITUDE'].round(3)

accidents_speedcams = crashes_2017_18.merge(speed_df, how="inner", on=('LATITUDE', 'LONGITUDE'))
accidents_speedcams

,Crash Date,Crash Time,WEATHER_CONDITION,LIGHTING_CONDITION,STREET_NO,STREET_DIRECTION,STREET_NAME,FIRST_CRASH_TYPE,PRIM_CONTRIBUTORY_CAUSE,INJURIES_TOTAL,...,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LOCATION_y,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-10,6,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
1,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-09,1,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
2,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-08,8,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
3,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-07,3,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
4,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-06,1,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
5,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-05,5,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
6,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-04,7,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
7,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-03,5,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
8,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-02,6,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0
9,2018-12-30,13:25:00,SNOW,"DARKNESS, LIGHTED ROAD",8033,W,ADDISON ST,PARKED MOTOR VEHICLE,IMPROPER BACKING,0.0,...,2019-06-01,4,1.121638e+06,1.922907e+06,"(41.94487363802818, -87.82831694409394)",39.0,22254.0,17.0,413.0,19.0


In [80]:
accidents_speedcams.columns

Index(['Crash Date', 'Crash Time', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
       'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'FIRST_CRASH_TYPE',
       'PRIM_CONTRIBUTORY_CAUSE', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE',
       'LONGITUDE', 'LOCATION_x', 'Season', 'ADDRESS', 'CAMERA ID',
       'VIOLATION DATE', 'VIOLATIONS', 'X COORDINATE', 'Y COORDINATE',
       'LOCATION_y', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

In [88]:
# Select desired columns and filter on 2017 and 2018
accidents_speedcams = accidents_speedcams[['Crash Date', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'FIRST_CRASH_TYPE',
                                           'INJURIES_TOTAL', 'PRIM_CONTRIBUTORY_CAUSE', 'INJURIES_FATAL',
                                           'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE',
                                           'LONGITUDE', 'CAMERA ID', 'VIOLATIONS', 'VIOLATION DATE']]

accidents_speedcams = accidents_speedcams.loc[(accidents_speedcams['Crash Date'] >= pd.to_datetime("2017-01-01").date()) & 
                                        (accidents_speedcams['Crash Date'] <= pd.to_datetime("2018-12-31").date()), :]

accidents_speedcams

,Crash Date,STREET_NO,STREET_DIRECTION,STREET_NAME,FIRST_CRASH_TYPE,INJURIES_TOTAL,PRIM_CONTRIBUTORY_CAUSE,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CAMERA ID,VIOLATIONS,VIOLATION DATE
0,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,6,2019-06-10
1,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,1,2019-06-09
2,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,8,2019-06-08
3,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,3,2019-06-07
4,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,1,2019-06-06
5,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,5,2019-06-05
6,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,7,2019-06-04
7,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,5,2019-06-03
8,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,6,2019-06-02
9,2018-12-30,8033,W,ADDISON ST,PARKED MOTOR VEHICLE,0.0,IMPROPER BACKING,0.0,13,1,12,41.945,-87.828,CHI175,4,2019-06-01


In [92]:
# Group crashes by their proximity to speed cameras
accidents_cameras_group = accidents_speedcams.groupby(['CAMERA ID'])
accidents_cameras_count = acc_speed_group.nunique().sort_values(by='Crash Date', ascending=False)

accidents_cameras_count = accidents_cameras_count[['Crash Date']] 
accidents_cameras_count.head(10)

,Crash Date
CAMERA ID,
CHI109,92
CHI154,60
CHI165,60
CHI075,37
CHI166,37
CHI076,37
CHI164,37
CHI135,32
CHI156,29
